# Manipulacion de ficheros en formato CVS y HDFS5

In [154]:
import numpy as np
import tables as tb
import pandas as pd
import glob
import cProfile

Cargamos todos los ficheros con mediciones diarias de las estaciones de estados unidos. Para ello, recorremos todos los 
ficheros csv para incluirlos en el dataframe de trabajo

In [155]:
#%ls "D:\Formacion\Master CIFF\entorno_DS_Python\proyecto\data"
%cd "D:\Formacion\Master CIFF\entorno_DS_Python\proyecto\data"
#%ls
pr = cProfile.Profile()
pr.enable()
path =r'D:\Formacion\Master CIFF\entorno_DS_Python\proyecto\data' # use your path
allFiles = glob.glob(path + "/*.txt")
df = pd.DataFrame()
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0,converters={'YearMonthDay':str})
    list_.append(df)

frame = pd.concat(list_)
pr.disable()
pr.print_stats(sort='time')


D:\Formacion\Master CIFF\entorno_DS_Python\proyecto\data
         103469 function calls (103307 primitive calls) in 7.456 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       15    5.604    0.374    5.604    0.374 {method 'read' of 'pandas.parser.TextReader' objects}
        1    0.552    0.552    0.552    0.552 <ipython-input-155-289b96713c49>:10(<module>)
       15    0.370    0.025    0.370    0.025 parsers.py:1197(__init__)
        3    0.242    0.081    0.242    0.081 {numpy.core.multiarray.concatenate}
       81    0.234    0.003    0.234    0.003 {numpy.core.multiarray.empty}
       15    0.154    0.010    6.600    0.440 parsers.py:272(_read)
       30    0.142    0.005    0.381    0.013 internals.py:4168(_stack_arrays)
       15    0.046    0.003    0.046    0.003 {pandas.lib.isnullobj}
    26549    0.012    0.000    0.026    0.000 {isinstance}
    10817    0.009    0.000    0.009    0.000 {getattr}
      750    0

Hacemos una copia del dataframe y añadimos una columna para facilitar el filtrado y estructuracion de la informacion en carpetas
además de poder utilizar la variable para agrupar por año.

In [156]:
workframe = frame[['YearMonthDay','Tmax','Tmin','Depart','PrecipTotal','StnPressure','AvgSpeed','Max5Speed','Water1']].copy()
workframe.head(5)
workframe['YearMonthDay'].apply(str)
workframe['YearMonthDay'].dtype
s = workframe.YearMonthDay.str[:4]
workframe['Year'] = s
workframe.head(5)

,YearMonthDay,Tmax,Tmin,Depart,PrecipTotal,StnPressure,AvgSpeed,Max5Speed,Water1,Year
0,20150101,41,37,M,0.05,29.75,5.3,M,M,2015
1,20150102,44,41,M,0.02,29.61,4.0,M,M,2015
2,20150103,62,38,M,0.01,29.55,3.9,M,M,2015
3,20150104,58,35,M,0.00,29.96,5.8,18,M,2015
4,20150105,52,30,M,0.00,30.11,3.7,M,M,2015


In [157]:
workframe.dtypes

YearMonthDay    object
Tmax            object
Tmin            object
Depart          object
PrecipTotal     object
StnPressure     object
AvgSpeed        object
Max5Speed       object
Water1          object
Year            object
dtype: object

Realizar los siguientes análisis
- Día en que ha habido más precipitaciones.

- Año en que ha habido más precipitaciones (obteniendo la media de cada año)

In [159]:
#Necesario eliminar unos outliers con datos incorrectos que detectamos en el dataframe
workframe = workframe[workframe['PrecipTotal'].str.contains("0.")]
workframe['PrecipTotal'] = workframe['PrecipTotal'].apply(pd.to_numeric)

In [160]:
#Obtenemos el maximo dia en que ha habido mas precipitaciones
workframe[workframe['PrecipTotal']==max(workframe['PrecipTotal'])]

,YearMonthDay,Tmax,Tmin,Depart,PrecipTotal,StnPressure,AvgSpeed,Max5Speed,Water1,Year
49639,20160509,M,M,M,30.23,29.56,11.6,18,M,2016


In [152]:
#Agrupamos por año y vemos la mayor media, en este caso la del 2016
grouped = workframe.groupby('Year').mean()
grouped.head(5)

,PrecipTotal
Year,
2015,0.069849
2016,0.072422


Creamos el almacenamiento, es esta caso utilizando pyTables, que es una implementación de python sobre la libreria de HDF5

In [94]:
h5file = pd.HDFStore("temp_events.h5")

In [95]:
print(h5file)

<class 'pandas.io.pytables.HDFStore'>
File path: temp_events.h5
Empty


Generamos los subsets de dataframe y volcamos cada subconjunto dentro de diferentes carpetas del directorio raiz. Podriamos 
desgranas mas la estructura de subdirectorios y añadir metainformación, pero el objetivo es medir el ratio de comportamiento 
entre ambos formatos

In [96]:
df2015 = workframe[workframe['Year']=='2015']
df2016 = workframe[workframe['Year']=='2016']
h5file['2015'] = df2015
h5file['2016'] = df2016

In [97]:
print(h5file)

<class 'pandas.io.pytables.HDFStore'>
File path: temp_events.h5
/2015            frame        (shape->[1,10])
/2016            frame        (shape->[1,10])


Volcamos el mismo dataset a un fichero csv para comparar

In [98]:
workframe.to_csv('temp_events.csv')

In [99]:
%ls -l temp_events.*

 Volume in drive D is DATA
 Volume Serial Number is 785D-DAFE

 Directory of D:\Formacion\Master CIFF\entorno_DS_Python\proyecto\data


 Directory of D:\Formacion\Master CIFF\entorno_DS_Python\proyecto\data

12/10/2016  18:48        38.468.999 temp_events.csv
12/10/2016  18:36             1.064 temp_events.h5
               2 File(s)     38.470.063 bytes
               0 Dir(s)  237.869.117.440 bytes free


Vemos que el ratio de compresion es brutal. Esto es debido a que hdf5 guarda la informacion en formato binario y comprimido, posibilitando ademas el acceso directo a subconjuntos de informacion. 

In [100]:
h5file.close()

Ahora verificaremos los tiempos de lectura del dataset, desde el fichero csv generado asi como del HDF5

In [168]:
pr.enable()
dataframe = pd.read_hdf('temp_events.h5', '2015')
dataframe2 = pd.read_hdf('temp_events.h5', '2016')
dataframe.merge(dataframe2)
#dataframe.head(5)
pr.disable()
pr.print_stats(sort='time')

         111416 function calls (111001 primitive calls) in 9.897 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       15    5.604    0.374    5.604    0.374 {method 'read' of 'pandas.parser.TextReader' objects}
       10    1.477    0.148    1.477    0.148 {cPickle.loads}
        1    0.552    0.552    0.552    0.552 <ipython-input-155-289b96713c49>:10(<module>)
       20    0.519    0.026    0.520    0.026 {method 'factorize' of 'pandas.hashtable.Factorizer' objects}
       15    0.370    0.025    0.370    0.025 parsers.py:1197(__init__)
        4    0.242    0.060    0.242    0.060 {numpy.core.multiarray.concatenate}
      118    0.234    0.002    0.234    0.002 {numpy.core.multiarray.empty}
       15    0.154    0.010    6.600    0.440 parsers.py:272(_read)
       30    0.142    0.005    0.381    0.013 internals.py:4168(_stack_arrays)
        2    0.134    0.067    0.134    0.067 {method '_read_array' of 'tables.hdf5ex

In [170]:
pr.enable()
df = pd.read_csv('temp_events.csv')
pr.disable()
pr.print_stats(sort='time')

         115119 function calls (114690 primitive calls) in 12.283 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       17    7.727    0.455    7.727    0.455 {method 'read' of 'pandas.parser.TextReader' objects}
       10    1.477    0.148    1.477    0.148 {cPickle.loads}
        1    0.552    0.552    0.552    0.552 <ipython-input-155-289b96713c49>:10(<module>)
       20    0.519    0.026    0.520    0.026 {method 'factorize' of 'pandas.hashtable.Factorizer' objects}
       17    0.421    0.025    0.422    0.025 parsers.py:1197(__init__)
      126    0.301    0.002    0.301    0.002 {numpy.core.multiarray.empty}
        4    0.242    0.060    0.242    0.060 {numpy.core.multiarray.concatenate}
       17    0.210    0.012    8.960    0.527 parsers.py:272(_read)
       34    0.199    0.006    0.506    0.015 internals.py:4168(_stack_arrays)
        2    0.134    0.067    0.134    0.067 {method '_read_array' of 'tables.hdf5e

Se aprecia como la lectura del csv es mas lenta, unos dos segundos respecto al mismo volumen de informacion (unas 900000 filas)

La siguiente porcion de código muestra una manera mas elaborada de manjear la informacion y volvarla registro a registro dentro
de una estructura de clase en HDF5. Se incluye simplemente como ilustración de como se podria realizar.

In [ ]:
def add_h5table(event, df): 
    for i in range(len(df)):
         event['YearMonthDay']  = df['YearMonthDay'][i]
         event['Tmax'] = df['Tmax'][i]
         event['Tmin'] = df['Tmin'][i]
         event['Depart'] = df['Depart'][i]
         event['PrecipTotal'] = df['PrecipTotal'][i]
         event['StnPressure'] = df['StnPressure'][i]
         event['AvgSpeed'] = df['AvgSpeed'][i]
         event['Max5Speed']= df['Max5Speed'][i]
         event['Water1']= df['Water1'][i]
         event['Year']= df['Year'][i]
         # Insert a new particle record
         event.append()
         return event
        
from tables import IsDescription, StringCol, Int64Col, UInt16Col,Float32Col
class Event(IsDescription):
     YearMonthDay = StringCol(8)   
     Tmax = UInt16Col()      
     Tmin = UInt16Col() 
     Depart = StringCol(1)
     PrecipTotal = Float32Col()
     StnPressure = Float32Col()    
     AvgSpeed = Float32Col()
     Max5Speed = StringCol(2)
     Water1 = StringCol(2)
     Year = UInt16Col()            
    
h5file = tb.open_file("temp_events2.h5", "w", title="Eventos meteorologicos entre 2015 y 2016")        
group2015 = h5file.create_group("/", '2015', 'Informacion meteorológica 2015')
table2015 = h5file.create_table(group2015, 'data', Event, 'events from dataframe')
event = table2015.row